In [ ]:
import numpy as np
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Introduction.

### 1.1. First, we read the csv documents provided by the company and see what is in each one.

In [ ]:
pd_sales=pd.read_csv('/content/drive/MyDrive/data_dsmarket/item_sales.csv',sep=',')
print("The shape of the sales datataframe is :", pd_sales.shape)

The shape of the sales datataframe is : (30490, 1920)


In [ ]:
pd_calendar=pd.read_csv('/content/drive/MyDrive/data_dsmarket/daily_calendar_with_events.csv',sep=',')
print("The shape of the calendar datataframe is :", pd_calendar.shape)

The shape of the calendar datataframe is : (1913, 5)


In [ ]:
pd_prices=pd.read_csv('/content/drive/MyDrive/data_dsmarket/item_prices.csv',sep=',')
print("The shape of the prices datataframe is :", pd_prices.shape)

The shape of the prices datataframe is : (6965706, 5)


### 1.2 We create our dataframe to be able to work with it through these 3 csv's.

#### First we order the sales table, putting the days and sales in vertical order.

In [ ]:
lista_dias=[]
lista_descripcion=[]
for i in pd_sales:
    if i.startswith('d_'):
      lista_dias.append(i)
    else:
      lista_descripcion.append(i)

pd_sales = pd.melt(pd_sales, id_vars=lista_descripcion, value_vars=lista_dias, var_name='d', value_name='n_sales')

In [ ]:
pd_sales

,id,item,category,department,store,store_code,region,d,n_sales
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
...,...,...,...,...,...,...,...,...,...
58327365,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,1
58327366,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0
58327367,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,0
58327368,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,d_1913,3


#### We sort and merge the calendar table and the prices table.

In [ ]:
from datetime import datetime, timedelta
def weekyearnum(dt):
    return dt.strftime("%Y%W")

def myweeyearknum(dt):
    offsetdt = dt + timedelta(days=+2);
    return weekyearnum(offsetdt)

def weeknum(dt):
    return dt.isocalendar()[1]

def myweeknum(dt):
    offsetdt = dt + timedelta(days=+2);
    return weeknum(offsetdt)

In [ ]:
pd_calendar['date'] = pd.to_datetime(pd_calendar['date'], format = "%Y-%m-%d")

pd_calendar['yearweek'] = pd_calendar['date'].apply(lambda x: myweeyearknum(x))

In [ ]:
pd_calendar['yearweek'] = pd_calendar['yearweek'].astype(float)
pd_merge = pd.merge(pd_prices,pd_calendar[['date','d','yearweek']], how="left",
                   on="yearweek")

In [ ]:
pd_merge.shape

(46271877, 7)

#### We join the three tables together and we are left with the columns we want.

In [ ]:
pd_completo = pd.merge(pd_sales[['id','item','category','department','store_code','region','d','n_sales']],pd_merge[['item','category','store_code','yearweek','date','d','sell_price']], how="left",
                   on=['item','category','store_code','d'])

In [ ]:
pd_completo.isnull().sum()

,0
id,0
item,0
category,0
department,0
store_code,0
region,0
d,0
n_sales,0
yearweek,12299413
date,12299413


### 1.3 Handling of nulls.

#### To avoid null values in the date column, which correspond to days on which the product was not sold, we assign each day in column d to a specific day in order.

In [ ]:
unique_d = pd_completo['d'].unique()

start_date = pd.to_datetime('2011-01-29')
dates = pd.date_range(start=start_date, periods=len(unique_d), freq='D')

date_mapping = pd.DataFrame({
    'd': unique_d,
    'date': dates
})

df = pd.merge(pd_completo[['id',
 'item',
 'category',
 'department',
 'store_code',
 'region',
 'd',
 'n_sales',
 'sell_price']], date_mapping, on='d', how='left')

#### The nulls in the sell_price column correspond to days where the product was not sold, we fill them with a backfill where the null value is completed with the previous price of that id.

In [ ]:
df['sell_price'] = df.groupby('id')['sell_price'].bfill()

In [ ]:
df.drop('d', axis = 1, inplace = True)

In [ ]:
df['revenue'] = df['sell_price']*df['n_sales']

In [ ]:
df.head()

,id,item,category,department,store_code,region,n_sales,sell_price,date,revenue
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,NYC_1,New York,0,12.74,2011-01-29,0.00
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,NYC_1,New York,0,5.28,2011-01-29,0.00
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,NYC_1,New York,0,3.95,2011-01-29,0.00
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,NYC_1,New York,0,5.77,2011-01-29,0.00
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,NYC_1,New York,0,3.96,2011-01-29,0.00


In [ ]:
df.isnull().sum()

,0
id,0
item,0
category,0
department,0
store_code,0
region,0
n_sales,0
sell_price,0
date,0
revenue,0


In [ ]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store_code  object        
 5   region      object        
 6   n_sales     int64         
 7   sell_price  float64       
 8   date        datetime64[ns]
 9   revenue     float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 4.3+ GB


#### We export our dataframe to work with it in Power BI.

In [ ]:
df.describe()

,n_sales,sell_price,date,revenue
count,"58,327,370.00","58,327,370.00",58327370,"58,327,370.00"
mean,1.13,5.56,2013-09-11 00:00:00.000000768,3.96
min,0.00,0.01,2011-01-29 00:00:00,0.00
25%,0.00,2.62,2012-05-21 00:00:00,0.00
50%,0.00,4.18,2013-09-11 00:00:00,0.00
75%,1.00,7.18,2015-01-02 00:00:00,3.63
max,763.00,134.15,2016-04-24 00:00:00,"2,597.18"
std,3.87,4.55,NaN,11.83


In [ ]:
df.describe(include='object')

,id,item,category,department,store_code,region
count,58327370,58327370,58327370,58327370,58327370,58327370
unique,30490,3049,3,7,10,3
top,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,NYC_1,New York
freq,1913,19130,27489810,15743990,5832737,23330948


In [ ]:
df.to_csv('df_ds_market.csv', sep=';', index=False)

## 2. Unsold products in the last 28 days and dataframe cleaning.

---



#### First we filtered the last 28 days.



In [ ]:
df_last_month = df[df['date']>'2016-03-27']

#### We are given a list of items that have not been sold in any of the 10 stores in the last 28 days. The idea is to give this list to the sales department and assess the value of removing these items from stock.

In [ ]:
productos_no_vendidos_ultimo_mes = df_last_month.groupby('id').filter(lambda x: x['n_sales'].sum() == 0)

In [ ]:
item_counts = productos_no_vendidos_ultimo_mes['item'].value_counts()

items_a_eliminar = item_counts[item_counts == 280].index

item_no_vendidos = productos_no_vendidos_ultimo_mes[productos_no_vendidos_ultimo_mes['item'].isin(items_a_eliminar)]

In [ ]:
item_no_vendidos['item'].unique()

array(['ACCESORIES_1_335', 'ACCESORIES_2_110', 'HOME_&_GARDEN_1_209',
       'HOME_&_GARDEN_1_366', 'HOME_&_GARDEN_2_158',
       'HOME_&_GARDEN_2_202', 'HOME_&_GARDEN_2_210',
       'HOME_&_GARDEN_2_456', 'HOME_&_GARDEN_2_502', 'SUPERMARKET_1_004',
       'SUPERMARKET_1_043', 'SUPERMARKET_1_120', 'SUPERMARKET_1_126',
       'SUPERMARKET_2_292', 'SUPERMARKET_3_002', 'SUPERMARKET_3_008',
       'SUPERMARKET_3_073', 'SUPERMARKET_3_077', 'SUPERMARKET_3_205',
       'SUPERMARKET_3_210', 'SUPERMARKET_3_271', 'SUPERMARKET_3_419',
       'SUPERMARKET_3_441', 'SUPERMARKET_3_444', 'SUPERMARKET_3_647'],
      dtype=object)

In [ ]:
lista_items_a_eliminar = item_no_vendidos['item'].unique()

#### We will remove this list of items from our dataframe from now on, as we will not be using these items in either the cluster or the time series.

In [ ]:
df_sin_no_vendidos_ultimo_mes = df.loc[~df['item'].isin(lista_items_a_eliminar)]

In [ ]:
# Once we have achieved our dataframe without the products we can consider to remove from our stock, we can match it to df to better manage our dataset.
df = df_sin_no_vendidos_ultimo_mes